In [179]:
import os

print(os.getcwd())

/home/urmzd/Documents/linear-genetic-programming/notebooks


In [180]:
import json
import pathlib
import pandas as pd
import seaborn as sns

from typing import Any, Dict, List

import numpy as np
import matplotlib.pyplot as plt
"""

    Structure:

    generations -> programs (sorted) -> [initial_states] -> [score]

    id: [
        {
            intial_state,
            q_table,
            score
        }
    ]

"""
def get_data_from_logs(log_file_path):
    with open(log_file_path, "r") as f:
        programs: Dict[str, List[Any]] = {}
        line_no = 0
        while line := f.readline():
            line_no += 1
            # print(line_no)
            # we've encountered a new generation
            if "generation" in line:
                # output programs in previous generation
                if len(programs):
                    yield programs
                # reset
                programs = {}
                continue
            else:
                program_entry = json.loads(line) 
                id = program_entry["id"] 

                programs.setdefault(id, [])

                programs[id].append(
                    {
                        "score": program_entry["score"],
                        "q_table": program_entry["q_table"]["table"],
                        "initial_state": program_entry["initial_state"] 
                    }
                )

        yield programs
    

path = pathlib.Path(os.getcwd()).parent / "assets/logs/test.log"
assert path.exists()

logs = list(get_data_from_logs(path))

assert len(logs) == 100

"""
What do we want to do here?
   1. Plot the scores of the best, worst and median ranked individuals as a function of their intitial_states
        a. In other words, given an initial state, how long does it take for the individual to accomplish their goal?
        b. We do this to figure out whether or not the intitial state is the reason for the stagnation we see (programs don't score more than ~100 episodes)

    Outstanding question here: is this comparison valid given that Q tables hold state?
        - to be clear, if the q table plays a critical role in the fitness assessement, and each score is derived
        from initial states with different q tables, does it make sense to say that program P performed better with initial state S
        when a previous intial state S' had less info overall? 
        - Should we clone the q_table when testing it with different initial states and set the q_table to the updated table derived during the median score? 
        - Also, should every generation have a new set of intial states?
"""
def convert_logs_to_dataframe(generations):

    bins = np.linspace(-0.4, -0.6)

    score_state_pairs = []

    data = []

    for generation_idx in range(len(generations)):

        # str, [{score, intial_state, q_table}]
        for id, program_entries in generations[generation_idx].items():
            for entry in program_entries:
                data.append({
                    "id": id,
                    "score": entry["score"],
                    "generation": generation_idx
                })
    
    return pd.DataFrame.from_records(data)


# df.groupby(["generation"])["fitness"].rank("max")


df = convert_logs_to_dataframe(logs)

# TODO: set fitness
df["fitness"] = df.groupby(["generation", "id"])["score"].transform("median")

# TODO: create rankings using fitness
df.groupby(["generation", "id"])["fitness"].describe().query("count > 5")

count   mean  std    min  \
generation id                                                               
99         01d2e71d-21fa-49a4-be59-324273e17bc8   10.0 -200.0  0.0 -200.0   
           075032a7-43af-47f2-90a5-e831381886b5   10.0 -200.0  0.0 -200.0   
           085e3dbd-b1ad-4ede-b434-722707364cf2   10.0 -200.0  0.0 -200.0   
           09aa06ac-be9d-4e42-bdee-3d3664514980   10.0 -200.0  0.0 -200.0   
           0b673ddf-5c86-4045-b850-78040d8ffa0e   10.0 -200.0  0.0 -200.0   
           0bdedd38-b2ae-4fa0-843f-82c19ac6bfb4   10.0 -200.0  0.0 -200.0   
           0c5a2c60-be35-42d9-8a17-0992ba122281   10.0 -200.0  0.0 -200.0   
           0cb58b91-e9a0-4fff-8db8-d639c8357955   10.0 -200.0  0.0 -200.0   
           0d1856bb-a191-4594-81bf-e1228700fc45   10.0 -200.0  0.0 -200.0   
           0e489d11-aac0-49f5-bf50-af6170abf943   10.0 -200.0  0.0 -200.0   
           11c3ec23-8475-4240-9252-c93e9134c4ac   10.0 -119.5  0.0 -119.5   
           1a504ca2-d0f8-4280-a99e-b8f6ba0f5ff8   10.0 -200.0  0.0 -200.0   
           1a89a6fb-fc10-4b7c-91af-9d56f83aa8f3   10.0 -200.0  0.0 -200.0   
           1f54d45f-73ab-4420-b9aa-955954c464fa   10.0 -200.0  0.0 -200.0   
           22f35a19-6c0a-42ae-8a74-819c910ae3eb   10.0 -200.0  0.0 -200.0   
           23aece16-90a2-4a3d-90c3-dd031c72e1a3   10.0 -200.0  0.0 -200.0   
           2753783c-8e38-4ca2-b630-f8110998fe41   10.0 -200.0  0.0 -200.0   
           289689d3-7158-4fc8-ab42-1f7fc819c938   10.0 -200.0  0.0 -200.0   
           30fcfc7c-f5d0-4733-b560-1b1ed38fbb7f   10.0 -200.0  0.0 -200.0   
           3502d9f6-dc15-4a64-a0bb-c39728f680c5   10.0 -200.0  0.0 -200.0   
           43012563-a02a-4f93-b1e9-828808e6441e   10.0 -200.0  0.0 -200.0   
           4be8a734-e489-48d3-b2d6-9b2efda46bc4   10.0 -200.0  0.0 -200.0   
           4f5409e6-ca4c-46e6-ae92-0b95d8c5a3d9   10.0 -200.0  0.0 -200.0   
           590275c5-bf94-4ee9-a1a3-fb74a67bffb3   10.0 -200.0  0.0 -200.0   
           5cda51bc-8719-4e19-934c-8bd1a4b59310   10.0 -200.0  0.0 -200.0   
           5f06a205-a13b-4de5-bc36-dc33c57a940f   10.0 -200.0  0.0 -200.0   
           60d15b10-1214-45c8-8681-696a44d73de5   10.0 -200.0  0.0 -200.0   
           60f1d943-e34a-46c1-9e74-3821a17f7773   10.0 -200.0  0.0 -200.0   
           6897eaf4-6da5-4975-bd42-09169aea932c   10.0 -200.0  0.0 -200.0   
           6f871308-5ec9-4cff-b602-859765715e6c   10.0 -200.0  0.0 -200.0   
           73cde70e-6d77-4793-a497-658dfa8def61   10.0 -200.0  0.0 -200.0   
           8812c34b-d751-409b-8ee1-ec080cd3b5c7   10.0 -200.0  0.0 -200.0   
           8b916f65-4981-4ffc-855d-7d3aaba30833   10.0 -200.0  0.0 -200.0   
           913d7a85-b82a-4e53-bb06-0bc8a97cd2e1   10.0 -200.0  0.0 -200.0   
           9b88e4b9-7151-406d-a25b-3182d0ecdd6b   10.0 -200.0  0.0 -200.0   
           aea767ee-0c4b-4dee-88cb-afdbb01b7cf2   10.0 -200.0  0.0 -200.0   
           b133e758-5df0-46d5-b87b-755ad8463834   10.0 -200.0  0.0 -200.0   
           b13f9e19-5688-4c57-bd73-d19f2daf46c1   10.0 -200.0  0.0 -200.0   
           b9650a95-fd16-4110-9255-357c733d0146   10.0 -200.0  0.0 -200.0   
           c21b0d89-11d6-4ea8-bd10-161ad1086f7c   10.0 -200.0  0.0 -200.0   
           c36d0731-b0e4-472d-b4a0-6dcc6ce891a0   10.0 -200.0  0.0 -200.0   
           c62b2b48-3cc2-4105-9070-8e0619f17d9b   10.0 -200.0  0.0 -200.0   
           d241fdb8-3bf2-41a9-80d4-4d75e2721d0e   10.0 -200.0  0.0 -200.0   
           db359d47-a6ed-49f5-b026-9a1465664e6c   10.0 -200.0  0.0 -200.0   
           df4b2926-a703-4edb-88fa-ae9af05b3e27   10.0 -200.0  0.0 -200.0   
           efa5acc4-7ea0-42c9-ac31-7adcdde23624   10.0 -200.0  0.0 -200.0   
           f2b19bae-7fc4-43a4-a77d-0ddd5eddd967   10.0 -200.0  0.0 -200.0   
           f3af9992-a3b2-4420-a3f3-01caef10e818   10.0 -200.0  0.0 -200.0   
           fb5a5506-0c62-40ab-bf53-724d3cce8857   10.0 -200.0  0.0 -200.0   
           fdb6049b-50a9-4633-b609-d2942e03892b   10.0 -200.0  0.0 -200.0   

                                            